# Assignation des projets fil rouge

Par représentation du problème sous contraintes, résolution avec ortools :
https://developers.google.com/optimization/assignment/overview

---
## Description du problème

Chaque élève doit être assigné à un projet.

Chaque projet à au plus 4 élèves.

Pour un élève est collecté ses préférences, que l'on pondère, le 1er choix pesant plus lourd que le second, etc. 

L'objectif est d'assigner élèves et projets en maximisant le poids total des préférences.

In [1]:
# import
import numpy as np
from ortools.sat.python import cp_model

Choix des poids associés aux préférences.

In [2]:
# Associe à chaque choix un poids
# Choix arbitraire d'une suite de Fibonacci
fibo = dict({1:89, 2:55, 3:34, 4:21, 5:13, 6:8, 7:5, 8:3, 9:2, 10:1, 11:0})
# Comment représenter l'absence de choix, 0 ?

In [14]:
# Paramètres
num_workers = 40
num_tasks = 11
# Nombre maximal d'élèves pour un projet
task_max = 4

In [15]:
# Initialisation au pif des préférences de chacun pour tester
prefs = []
for i in range(num_workers):
    arr = np.arange(1, num_tasks+1)
    np.random.shuffle(arr)
    prefs.append(arr.tolist())
    
costs = [[fibo[worker_pref] for worker_pref in c_worker] for c_worker in prefs]
costs

[[34, 3, 21, 8, 89, 2, 1, 55, 0, 5, 13],
 [13, 5, 21, 3, 34, 2, 8, 55, 0, 89, 1],
 [1, 8, 89, 5, 55, 3, 2, 21, 0, 13, 34],
 [2, 21, 13, 5, 55, 89, 34, 8, 0, 3, 1],
 [13, 55, 89, 5, 8, 34, 0, 3, 2, 1, 21],
 [0, 89, 1, 3, 5, 8, 34, 55, 2, 13, 21],
 [13, 21, 1, 8, 0, 34, 55, 89, 3, 2, 5],
 [2, 21, 55, 1, 13, 89, 3, 5, 34, 8, 0],
 [34, 0, 1, 3, 21, 5, 55, 89, 13, 8, 2],
 [0, 8, 89, 55, 21, 13, 34, 2, 1, 5, 3],
 [34, 1, 5, 13, 2, 3, 55, 21, 8, 0, 89],
 [5, 3, 13, 55, 8, 34, 0, 89, 2, 21, 1],
 [5, 34, 8, 89, 55, 1, 3, 21, 0, 2, 13],
 [0, 1, 89, 3, 13, 34, 8, 5, 55, 2, 21],
 [34, 0, 2, 5, 13, 55, 1, 89, 8, 21, 3],
 [34, 1, 21, 55, 3, 2, 13, 8, 89, 5, 0],
 [1, 55, 34, 8, 5, 2, 13, 89, 0, 3, 21],
 [2, 55, 13, 89, 3, 21, 5, 1, 8, 0, 34],
 [5, 55, 0, 13, 2, 3, 34, 8, 89, 1, 21],
 [2, 0, 8, 89, 5, 55, 21, 1, 34, 3, 13],
 [0, 2, 1, 13, 5, 21, 55, 3, 34, 89, 8],
 [13, 0, 1, 89, 2, 8, 21, 3, 34, 5, 55],
 [0, 89, 13, 8, 2, 1, 34, 5, 21, 55, 3],
 [1, 5, 8, 55, 13, 21, 2, 34, 89, 3, 0],
 [5, 0, 55, 34, 

In [7]:
# Déclare le modèle
model = cp_model.CpModel()

In [8]:
# Déclare les variables
x = {}
for worker in range(num_workers):
    for task in range(num_tasks):
        x[worker, task] = model.NewBoolVar(f'x[{worker},{task}]')

In [9]:
# Déclare les contraintes

# Contraint chaque élève doit avoir exactement 1 projet.
for worker in range(num_workers):
    model.AddExactlyOne(x[worker, task] for task in range(num_tasks))

# Contraint chaque projet doit avoir au maximum task_max élèves.
t_workers = []
for task in range(num_tasks):
    sub_t_workers = []
    for worker in range(num_workers):
            sub_t_workers.append(x[worker, task])
    t_workers.append(sub_t_workers)

for e in t_workers:
    model.Add(sum(e) <= task_max)

In [10]:
# Déclare l'objectif
objective_terms = []
for worker in range(num_workers):
    for task in range(num_tasks):
        objective_terms.append(costs[worker][task] * x[worker, task])
model.Maximize(sum(objective_terms))

In [11]:
# Déclare le solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [12]:
# Affiche les solutions si trouvées
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Total cost = {solver.ObjectiveValue()}\n')
    for worker in range(num_workers):
        for task in range(num_tasks):
            if solver.BooleanValue(x[worker, task]):
                print(f'Worker {worker} assigned to task {task}.' +
                      f' Cost = {costs[worker][task]}')
else:
    print('No solution found.')

Total cost = 3322.0

Worker 0 assigned to task 3. Cost = 55
Worker 1 assigned to task 5. Cost = 89
Worker 2 assigned to task 10. Cost = 89
Worker 3 assigned to task 8. Cost = 89
Worker 4 assigned to task 6. Cost = 89
Worker 5 assigned to task 10. Cost = 89
Worker 6 assigned to task 2. Cost = 89
Worker 7 assigned to task 10. Cost = 89
Worker 8 assigned to task 8. Cost = 89
Worker 9 assigned to task 3. Cost = 89
Worker 10 assigned to task 7. Cost = 55
Worker 11 assigned to task 6. Cost = 89
Worker 12 assigned to task 4. Cost = 89
Worker 13 assigned to task 2. Cost = 89
Worker 14 assigned to task 9. Cost = 55
Worker 15 assigned to task 5. Cost = 89
Worker 16 assigned to task 6. Cost = 89
Worker 17 assigned to task 7. Cost = 89
Worker 18 assigned to task 5. Cost = 89
Worker 19 assigned to task 9. Cost = 55
Worker 20 assigned to task 8. Cost = 89
Worker 21 assigned to task 2. Cost = 89
Worker 22 assigned to task 8. Cost = 89
Worker 23 assigned to task 1. Cost = 89
Worker 24 assigned to task